## seasonal monthly forecast from Copernicus
see https://cds.climate.copernicus.eu/cdsapp#!/dataset/seasonal-postprocessed-single-levels?tab=overview
for information

### download SEAS5 single level t2m anomaly

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'format': 'netcdf',
        'originating_centre': 'ecmwf',
        'product_type': 'ensemble_mean',
        'variable': '2m_temperature_anomaly',
        'year': '2022',
        'month': '10',
        'leadtime_month': [
            '1', '2', '3',
            '4', '5', '6',
        ],
        'system': '5',
    },
    'SEAS5_202210_2m_temperature_anomaly.nc')

### download single members of seasonal forecast

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'format': 'netcdf',
        'originating_centre': 'ecmwf',
        'product_type': 'monthly_mean',
        'variable': '2m_temperature_anomaly',
        'year': '2023',
        'month': '6',
        'leadtime_month': [
            '1', '2', '3',
            '4', '5', '6',
        ],
        'system': '51',
    },
    'SEAS5_202210_2m_temperature_anomaly_members.nc')

### download UKMO single level t2m anomaly

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'format': 'netcdf',
        'originating_centre': 'ncep',
        'product_type': 'ensemble_mean',
        'variable': '2m_temperature_anomaly',
        'year': '2022',
        'month': '10',
        'leadtime_month': [
            '1', '2', '3',
            '4', '5', '6',
        ],
        'system': '2',
    },
    'NCEP_202210_2m_temperature_anomaly.nc')

### plot map¶

In [ ]:
import xarray as xr

with xr.open_dataset('../example_data/SEAS5_202210_2m_temperature_anomaly.nc') as ds:
    ds.isel(time=0)['t2a'].plot()

### plot map with borders

In [ ]:
import cartopy.crs as ccrs
import cartopy.features as cf
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))

with xr.open_dataset('../example data/SEAS5_202210_2m_temperature_anomaly.nc') as ds:
    ds.isel(time=0)['t2a'].plot(ax=ax)

ax.coastlines()

### zoom in to Europe

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))

with xr.open_dataset('SEAS5_202210_2m_temperature_anomaly.nc') as ds:
    ds.isel(time=3)['t2a'].plot(vmax=2, vmin=-2, cmap='RdBu_r')

ax.add_feature(cf.COASTLINE)
ax.add_feature(cf.BORDERS)

ax.set_xlim(-20,30)
ax.set_ylim(30,70)

### plot also for NCEP

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))

with xr.open_dataset('NCEP_202210_2m_temperature_anomaly.nc') as ds:
    ds.isel(time=3)['t2a'].plot(vmax=2, vmin=-2, cmap='RdBu_r')

ax.add_feature(cf.COASTLINE)
ax.add_feature(cf.BORDERS)

ax.set_xlim(-20,30)
ax.set_ylim(30,70)

### difference between NCEP and SEAS5

In [ ]:
ds_ncep=xr.open_dataset('NCEP_202210_2m_temperature_anomaly.nc') 
ds_seas5=xr.open_dataset('SEAS5_202210_2m_temperature_anomaly.nc')

print(ds_seas5)
print(ds_ncep)

### regrid ncep to SEAS5 grid

In [ ]:
regridded_ds_ncep = ds_ncep.interp(coords={'latitude': ds_seas5.latitude.data, 'longitude': ds_seas5.longitude.data})

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))

regridded_ds_ncep.isel(time=3)['t2a'].plot(vmax=2, vmin=-2, cmap='RdBu_r')

ax.add_feature(cf.COASTLINE)
ax.add_feature(cf.BORDERS)

ax.set_xlim(-20,30)
ax.set_ylim(30,70)

In [ ]:
### 

In [ ]:



# Convert longitude from 0-360 to -180 to 180
ds_seas5 = ds_seas5.roll(longitude=(ds_seas5.dims['longitude'] // 2), roll_coords=True)
# Update the longitude values
ds_seas5['longitude'] = ((ds_seas5.longitude + 180) % 360) - 180


In [ ]:
regridded_ds_ncep = ds_ncep.interp(coords={'latitude': ds_seas5.latitude.data, 'longitude': ds_seas5.longitude.data})

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))

regridded_ds_ncep.isel(time=3)['t2a'].plot(vmax=2, vmin=-2, cmap='RdBu_r')

ax.add_feature(cf.COASTLINE)
ax.add_feature(cf.BORDERS)

ax.set_xlim(-20,30)
ax.set_ylim(30,70)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))

(regridded_ds_ncep-ds_seas5).isel(time=0)['t2a'].plot(vmax=2, vmin=-2, cmap='RdBu_r')

ax.add_feature(cf.COASTLINE)
ax.add_feature(cf.BORDERS)

ax.set_xlim(-20,30)
ax.set_ylim(30,70)

### multi member forecast

In [ ]:
with xr.open_dataset('SEAS5_202306_2m_temperature_anomaly_members.nc') as ds:
    ds

In [ ]:
import numpy as np
fig, ax = plt.subplots(2,2,figsize=(20, 15), subplot_kw=dict(projection=ccrs.PlateCarree()))

ax = ax.flatten()

for i in range(4):
    member= np.random.randint(0, 50)
    ds['t2a'].isel(dict(time=1, number=member)).plot(ax=ax[i], vmin=-4, vmax=4, cmap='RdBu_r')
    ax[i].add_feature(cf.COASTLINE)
    ax[i].add_feature(cf.BORDERS)

    ax[i].set_xlim(-20,30)
    ax[i].set_ylim(30,70)
    
# regridded_ds_ncep.isel(time=3)['t2a'].plot(vmax=2, vmin=-2, cmap='RdBu_r')



In [ ]:
import numpy as np
fig, ax = plt.subplots(figsize=(20, 15), subplot_kw=dict(projection=ccrs.PlateCarree()))

ds['t2a'].isel(time=1).quantile(0.9,dim='number').plot(ax=ax)
ax.add_feature(cf.COASTLINE)
ax.add_feature(cf.BORDERS)

ax.set_xlim(-20,30)
ax.set_ylim(30,70)

In [ ]:
# Select the data for the desired time step
data = ds['t2a'].isel(time=1).mean(dim='number')

# Create the contour line plot
fig, ax = plt.subplots(figsize=(20, 15), subplot_kw=dict(projection=ccrs.PlateCarree()))

ds['t2a'].isel(time=1).mean(dim='number').plot(ax=ax)

contour_lines = ax.contour(data.longitude, data.latitude, data, levels=[0], colors='black')

num_points = 10  # Increase this value for smoother contour lines

# Interpolate the contour line to make it smoother
path = contour_lines.collections[0].get_paths()[0]
vertices = path.vertices
interp_vertices = np.linspace(0, vertices.shape[0] - 1, num_points)
interp_line = np.stack([np.interp(interp_vertices, np.arange(vertices.shape[0]), vertices[:, 0]),
                        np.interp(interp_vertices, np.arange(vertices.shape[0]), vertices[:, 1])], axis=1)

# Plot the interpolated contour line
ax.plot(interp_line[:, 0], interp_line[:, 1], 'k-', linewidth=1)



ax.add_feature(cf.COASTLINE, color='gray')
ax.add_feature(cf.BORDERS, color='gray')

ax.set_xlim(-20,30)
ax.set_ylim(30,70)

### time series of location

In [ ]:
#lat_loc=52
#lon_loc=5
lat_loc=-6
lon_loc=260


fig, ax = plt.subplots(figsize=(10,10))
ds_interp = ds['t2a'].interp(latitude=lat_loc, longitude=lon_loc, method='linear')

for m in ds_interp.number:
    ds_interp.isel(number=m).plot(ax=ax, color='steelblue', linewidth=1, alpha=0.5)

ds_interp.mean(dim='number').plot(ax=ax, color='red', linewidth=2)

ds_interp.quantile(0.9,dim='number').plot(ax=ax, color='gray', linewidth=2)

ds_interp.quantile(0.1,dim='number').plot(ax=ax, color='gray', linewidth=2)

ax.fill_between(ds_interp.time,ds_interp.quantile(0.1,dim='number'), ds_interp.quantile(0.9,dim='number'), alpha=0.4)

ax.plot(ds_interp.time,[0,0,0,0,0,0], '--', color='black')


In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'seasonal-monthly-single-levels',
    {
        'format': 'grib',
        'originating_centre': 'ecmwf',
        'system': '51',
        'variable': '2m_temperature',
        'product_type': 'ensemble_mean',
        'year': '2023',
        'month': '03',
        'leadtime_month': '1',
    },
    'download_ec.grib')

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))

(fc_ec-fc_ukmo).t2m.plot(ax=ax)
ax.coastlines()

#### read UKMO

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'seasonal-monthly-single-levels',
    {
        'format': 'grib',
        'originating_centre': 'ukmo',
        'system': '601',
        'variable': '2m_temperature',
        'product_type': 'ensemble_mean',
        'year': '2023',
        'month': '02',
        'leadtime_month': '2',
    },
    'download_ukmo.grib')

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'seasonal-monthly-single-levels',
    {
        'format': 'grib',
        'originating_centre': 'ukmo',
        'system': '601',
        'variable': '2m_temperature',
        'product_type': [
            'ensemble_mean', 'hindcast_climate_mean',
        ],
        'year': '2023',
        'month': '02',
        'leadtime_month': '2',
    },
    'download_ukmo_clim.grib')

In [ ]:
import xarray as xr

In [ ]:
fc_hc = xr.open_dataset('download_ukmo_clim.grib',engine ="cfgrib",filter_by_keys={'dataType': 'hcmean'})
fc = xr.open_dataset('download_ukmo_clim.grib',engine ="cfgrib",filter_by_keys={'dataType': 'em'})

In [ ]:
fc - fc_hc

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))

(((fc-fc_hc)/fc_hc)*100.0)['t2m'].plot(ax=ax)

ax.coastlines()

In [ ]:
fc['msnsrf'].plot()

In [ ]:
fc_ec = xr.open_dataset('download_ec.grib')
fc_ukmo = xr.open_dataset('download_ukmo.grib')

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(projection=ccrs.PlateCarree()))

(fc_ec-fc_ukmo).t2m.plot(ax=ax)
ax.coastlines()

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'derived-near-surface-meteorological-variables',
    {
        'version': '2.1',
        'format': 'zip',
        'variable': 'surface_downwelling_shortwave_radiation',
        'reference_dataset': 'cru',
        'year': '2019',
        'month': '07',
    },
    'download.zip')

In [ ]:
fc = xr.open_dataset('SWdown_WFDE5_CRU_201907_v2.1.nc')

In [ ]:
fc.isel(time=0)['SWdown'].plot()

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'insitu-observations-surface-marine',
    {
        'format': 'csv-obs.zip',
        'year': '1851',
        'month': '01',
        'day': '01',
        'data_quality': 'passed',
        'variable': [
            'air_temperature', 'wind_speed',
        ],
    },
    'download.csv-obs.zip')

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'sis-shipping-consumption-on-routes',
    {
        'format': 'zip',
        'variable': [
            'fuel_consumption_at_fixed_shaft_power', 'fuel_consumption_at_fixed_speed', 'shaft_power_at_fixed_speed',
            'ship_speed_at_fixed_shaft_power', 'trip_duration_at_fixed_shaft_power',
        ],
        'product_type': 'historical',
        'statistic': 'daily',
        'departure_port': 'gibraltar',
        'arrival_port': 'new_york',
        'year': '2017',
    },
    'download.zip')

In [ ]:
fc = xr.open_dataset(r'./download/shaft_power_fixed_speed_historical_daily_product_tanker_from_gibraltar_to_new_york_2017_v0.0.nc')

In [ ]:
fc.isel(time=1).isel(speed=1)['shaft_power'].plot()

In [ ]:
fc.sel(lat=-7.01)

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'format': 'grib',
        'year': '2023',
        'originating_centre': 'ecmwf',
        'system': '51',
        'product_type': 'monthly_mean',
        'month': '03',
        'leadtime_month': [
            '1', '2', '3',
            '4', '5', '6',
        ],
        'area': [
            60, 0, 40,
            20,
        ],
        'variable': [
            '10m_wind_speed_anomaly', '2m_temperature_anomaly', 'surface_solar_radiation_anomalous_rate_of_accumulation',
        ],
    },
    'download.grib')

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'format': 'grib',
        'originating_centre': 'ukmo',
        'system': '601',
        'variable': '2m_temperature_anomaly',
        'product_type': 'monthly_mean',
        'year': '2023',
        'month': '02',
        'leadtime_month': [
            '1', '2', '3',
            '4', '5', '6',
        ],
        'area': [
            60, 0, 40,
            20,
        ],
    },
    'download_uk.grib')

In [ ]:
fc = xr.open_dataset('download_uk.grib')

In [ ]:
fc

In [ ]:
import matplotlib.pyplot as plt
lat = xr.DataArray([51.], dims="z")
lon = xr.DataArray([4.], dims="z")
df = fc.interp(latitude=lat,longitude=lon)

In [ ]:
fc['t2a'].isel(dict(number=0,)).plot()

In [ ]:
import matplotlib.pyplot as plt
lat = xr.DataArray([51], dims="z")
lon = xr.DataArray([4], dims="z")
df = fc.interp(latitude=lat,longitude=lon).to_dataframe().reset_index()#.pivot(columns=['number'],values=['t2a'])

fig,ax = plt.subplots(figsize=(10,5))
numbers = df.number.unique()

for n in numbers:
    ax.plot(df.loc[df.number==n]['t2a'].values)



In [ ]:
df

In [ ]:
import xarray as xr

# Maak een xarray-object met ontbrekende waarden
ds = xr.Dataset({
    'x': xr.DataArray([0, 1, 2, 3], dims='x'),
    'y': xr.DataArray([0, 1, 2, 3], dims='y'),
    'data': xr.DataArray([[1, 2, 3, 4], [5, 6, 7, float('nan')], [9, 10, float('nan'), 12], [13, 14, 15, 16]], dims=('y', 'x'))
})

# Interpoleer het xarray-object naar een nieuw rooster met ontbrekende waarden
new_coords = {'x': [0.5, 1.5, 2.5], 'y': [0.5, 1.5, 2.5]}
ds_interp = ds.fillna(ds.interp(new_coords))

# Bekijk het resultaat
print(ds_interp)